In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

data1 = pd.read_csv("/content/drive/MyDrive/data/twcs.csv")

In [4]:
data = data1.iloc[:500]

In [5]:
data.sample(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
470,775,115875,True,Tue Oct 31 21:59:51 +0000 2017,@Uber_Support I’m not getting your texts. http...,774,NaN
390,689,AmazonHelp,False,Tue Oct 31 21:54:32 +0000 2017,@115849 Please report this to our support team...,686,690.0
396,696,AppleSupport,False,Tue Oct 31 22:27:49 +0000 2017,@115854 We're here for you. Which version of t...,697,698.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0
198,287,AdobeCare,False,Thu Oct 26 19:03:22 +0000 2017,"@115776 Unfortunately, we are not able to repr...",285,288.0
149,235,ATVIAssist,False,Tue Oct 31 21:53:44 +0000 2017,@115763 Can you please give me your gamertag /...,236,237.0
277,384,115804,True,Tue Oct 31 22:08:05 +0000 2017,"@Morrisons That's pointless as it's every day,...",386,383.0
443,747,115865,True,Tue Oct 31 18:08:04 +0000 2017,Hola @AppleSupport necesito urgente la actuali...,746,NaN
280,385,115804,True,Tue Oct 31 21:28:45 +0000 2017,@Morrisons so fed up of this very familiar sig...,383,NaN
439,740,115865,True,Tue Oct 31 22:14:45 +0000 2017,@AppleSupport iPhone 7 Plus 😊,738,744.0


# removing HTML and different sorts of Weblinks

In [6]:
import re

def remove_html_and_links(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*>', '', text)

    # Remove web links
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # Remove Google search links
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text)

    return text

In [7]:
data.text = data.text.apply(remove_html_and_links)

<ipython-input-7-25455b4f398d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text = data.text.apply(remove_html_and_links)


# camel casing the words

In [8]:
def split_camel_case(word):
    # Use regular expression to split camel case words
    # This pattern matches where a lowercase letter is followed by an uppercase letter
    # or where an uppercase letter is followed by one or more lowercase letters
    parts = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', word)
    return ' '.join(parts)

In [9]:
data["text"] = data["text"].apply(split_camel_case)

<ipython-input-9-ba614e148c0e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(split_camel_case)


### The text contains the japanese language converting into English inorder understand the corpus clearly for the further study

In [10]:
pip install langdetect

In [11]:
 pip install googletrans

In [12]:
from langdetect import detect
from googletrans import Translator

def translate_japanese_sentences(text):
    translator = Translator()

    # Split text into sentences
    sentences = text.split('.')

    translated_text = []

    for sentence in sentences:
        # Detect language of the sentence
        try:
            lang = detect(sentence)
        except:
            # If language detection fails, assume it's English
            lang = 'en'

        # If the detected language is Japanese, translate to English
        if lang == 'ja':
            translated_sentence = translator.translate(sentence, src='ja', dest='en').text
            translated_text.append(translated_sentence)
        else:
            translated_text.append(sentence)

    # Join translated sentences back into a text
    translated_text = '. '.join(translated_text)

    return translated_text

# Example usage:
mixed_text = "This is an example sentence in English. これは日本語の例文です。 This is another English sentence."
translated_text = translate_japanese_sentences(mixed_text)
print("Translated text:", translated_text)

Translated text: This is an example sentence in English.  これは日本語の例文です。 This is another English sentence. 


In [13]:
data["text"] = data["text"].apply(translate_japanese_sentences)

<ipython-input-13-4322ca367f3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(translate_japanese_sentences)


# Lowercaseing the words and sentence

In [14]:
def lowercase_text(text):
    # Convert the text to lowercase
    lowercase_text = text.lower()

    return lowercase_text

data["text"] = data["text"].apply(lowercase_text)

<ipython-input-14-b7edae9414f1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(lowercase_text)


# Removing punctuations

In [15]:
import string
def remove_punctuations(text):
    # Using string.punctuation to get the set of punctuation characters
    punctuations = string.punctuation
    # Removing punctuations from the text
    text = ''.join(char for char in text if char not in punctuations)
    return text

In [16]:
data["text"] = data["text"].apply(remove_punctuations)

<ipython-input-16-6ac9b07c8459>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_punctuations)


# defing the chatwords and it's abbrevations and replacing them with there abbrevations

In [17]:
chatwordss = {"lol":"Laugh Out Loud",
"brb":"Be Right Back",
"btw":"By The Way",
"idk":"I Don't Know",
"imo":"In My Opinion",
"imho":"In My Humble Opinion",
"omg":"Oh My God",
"rofl":"Rolling On The Floor Laughing",
"ttyl":"Talk To You Later",
"omw":"On My Way",
"afk":"Away From Keyboard",
"b4":"Before",
"cu":"See You",
"jk":"Just Kidding",
"np":"No Problem",
"thx":"thanks",
"yw":"You're Welcome",
"wb":"welcome back",
"gg":"good game",
"gr8":"Great",
"hru":"how are you",
"tyt":"take your time",
"ic":"I see",
"plz":"please",
"sry":"sorry",
"ttyt":"talk to you tomorrow",
"wtf":"What The F***",
"wyd":"What You Doing",
"ily":"I Love You",
"afaik":"as far as I know",
"bfn":"bye for now",
"fwiw":"For What It's Worth",
"hth":"hope this helps",
"irl":"In Real Life",
"l8r":"Later",
"lmk":"let me know",
"rn":"right now",
"tbh":"To Be Honest",
"tmi":"Too Much Information",
"u":"you",
"ur":"your",
"w/e":"whatever",
"wru":"where are you",
"yolo":"You Only Live Once",
"gtg":"got to go",
"fomo":"fear of missing out",
"nsfw":"Not Safe For Work",
"smh":"Shaking My Head",
"tbt":"Throwback Thursday",
"tl;dr":"Too Long; Didn't Read",
"af":"as f***",
"bae":"before anyone else",
"fml":"f*** my life",
"ftw":"for the win",
"idc":"I Don't Care",
"idgaf":"I don't give a f***",
"lmao":"Laughing My A** Off",
"lmfao":"Laughing My F***ing A** Off",
"ofc":"of course",
"omfg":"Oh My F***ing God",
"stfu":"Shut The F*** Up",
"tfw":"that feeling when",
"ty":"Thank You",
"wym":"What You Mean",
"bruh":"bro",
"nvm":"Never Mind",
"jfc":"Jesus f***ing Christ",
"hbu":"how about you",
"ttfn":"ta ta for now",
"tldr":"too long didn't read",
"g2g":"got to go",
"hmu":"hit me up",
"dm":"Direct Message",
"rp":"roleplay",
"icymi":"In Case You Missed It",
"smfh":"shaking my f***ing head",
"lms":"like my status",
"asl":"age, sex, location",
"tf":"the f***",
"nbd":"no big deal",
"gtfo":"get the f*** out",
"tba":"to be announced",
"jic":"just in case",
"wth":"What The Hell",
"ffs":"for f***'s sake",
"pov":"point of view",
"srsly":"seriously",
"tyvm":"thank you very much",
"mt":"Modified Tweet",
"ff":"Follow Friday",
"brt":"Be Right There",
"icydk":"In Case You Didn't Know",
"hbd":"Happy Birthday",
"ht":"Hat Tip",
"ymmv":"Your Mileage May Vary",
"asap":"As Soon As Possible",
"iirc":"If I Remember Correctly",
"ootd":"Outfit Of The Day",
"ot":"Off Topic",
"tgif":"Thank God It's Friday",
"b4n":"Bye For Now",
"b4u":"Before You",
"b4u2":"Before You Too",
"cya":"See You",
"h8":"Hate",
"h9":"Hate",
"2day":"Today",
"2moro":"Tomorrow",
"2nite":"Tonight",
"2mrw":"Tomorrow",
"4get":"Forget",
"l8":"Late",
"10q":"Thank You",
"1ce":"Once",
"b4r":"Before",
"cuz":"Because",
"thnx":"Thanks",
"2c":"To See",
"2l8":"Too Late",
"2q2b4g10q":"To Be Or Not To Be Thankful",
"4eva":"Forever",
"4give":"Forgive",
"4gtn":"Forgotten",
"4gv":"Forgive",
"4nr":"Foreigner",
"rt":"Retweet"}

In [18]:
def remove_chatwords(text):
    new_text = []
    for w in text.split():
        if w in chatwordss:
            new_text.append(chatwordss[w])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [19]:
data["text"] = data["text"].apply(remove_chatwords)

<ipython-input-19-02fae6b06f1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_chatwords)


### may chatwords abbrevations contain the uppercase words or unnecessary puncutaions so removing them all once again

In [20]:
data.text = data.text.apply(lowercase_text)

<ipython-input-20-d27acafa5757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text = data.text.apply(lowercase_text)


In [21]:
data.text = data.text.apply(remove_punctuations)

<ipython-input-21-8d236a63bdba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text = data.text.apply(remove_punctuations)


# Removing the emoji's

In [22]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"  # Chinese and Japanese characters
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

    # Remove emojis from the text
    text = emoji_pattern.sub(r'', text)

    return text

In [23]:
data["text"] = data["text"].apply(remove_emojis)

<ipython-input-23-2a7c0ca92db4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_emojis)


# Removing the numbers which don't carry weight

In [24]:
import re

def remove_numbers(text):
    # Regular expression pattern to match numbers
    number_pattern = re.compile(r'\d+')
    # Replace numbers with an empty string
    text = number_pattern.sub('', text)
    return text

In [25]:
data["text"] = data["text"].apply(remove_numbers)

<ipython-input-25-c9de7656bb02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_numbers)


# Remvoing the Whitespace

In [26]:
def remove_extra_whitespace(text):
    # Split the text into words and remove empty strings (resulting from consecutive white spaces)
    words = text.split()
    # Join the words back together with a single space between them
    clean_text = ' '.join(words)
    return clean_text

In [27]:
data["text"] = data["text"].apply(remove_extra_whitespace)

<ipython-input-27-aa96f48376ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_extra_whitespace)


# spelling correction using the textblob

In [28]:
from textblob import TextBlob
def correct_words_with_textblob(text):
    # Use TextBlob for spell correction
    blob = TextBlob(text)
    corrected_text = str(blob.correct())
    return corrected_text

Since, It taking me lot of time to compile so i going to select the first 10000 sample apply, the methods on it.

In [29]:
data["text"] = data["text"].apply(correct_words_with_textblob)

<ipython-input-29-ffb675126979>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(correct_words_with_textblob)


In [30]:
data.sample(100)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,this is maddening to hear please shoot us a d ...,NaN,12.0
460,765,115870,True,Tue Oct 31 18:20:46 +0000 2017,after update is no specify on my lock screen a...,764,NaN
335,628,115826,True,Tue Oct 31 21:57:24 +0000 2017,how about you guns figure out my box one proje...,626,630.0
349,643,115834,True,Mon Oct 30 23:44:45 +0000 2017,bought an echo show and it won t recognize a s...,642,NaN
471,776,Uber_Support,False,Tue Oct 31 22:17:58 +0000 2017,for more into about user eats availability cli...,NaN,777.0
...,...,...,...,...,...,...,...
470,775,115875,True,Tue Oct 31 21:59:51 +0000 2017,user support m not getting your texts,774,NaN
339,633,AmazonHelp,False,Tue Oct 31 22:26:37 +0000 2017,m sorry for the wait please reach out to us so...,NaN,634.0
243,333,115797,True,Tue Oct 31 23:44:59 +0000 2017,air asia support hi have d you my booking and ...,335,332.0
307,601,115811,True,Tue Oct 31 16:13:35 +0000 2017,can connect my interactive broker canada with ...,600,NaN


# Stopword removal

In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
from nltk.corpus import stopwords

# Download NLTK stopwords list if not already downloaded
nltk.download('stopwords', quiet=True)

def remove_stopwords(text):
    # Get English stopwords from NLTK
    english_stopwords = set(stopwords.words('english'))
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in english_stopwords]
    # Join the filtered tokens back into a single string
    text = ' '.join(filtered_tokens)
    return text

In [36]:
data["text"] = data["text"].apply(remove_stopwords)

<ipython-input-36-44754f414dc9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(remove_stopwords)


# stemming

In [37]:
from nltk.stem import PorterStemmer
nltk.download('punkt', quiet=True)

def perform_stemming(text):
    # Initialize a Porter stemmer
    stemmer = PorterStemmer()
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Stem each token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # Join the stemmed tokens back into a single string
    stemmed_text = ' '.join(stemmed_tokens)
    return stemmed_text

In [38]:
data["text"] = data["text"].apply(perform_stemming)

<ipython-input-38-a3e5ab3bf734>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(perform_stemming)


In [39]:

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

def perform_lemmatization(text):
    # Initialize a WordNet lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Lemmatize each token
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    # Join the lemmatized tokens back into a single string
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

def get_wordnet_pos(word):
    # Map POS tag to first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Default to noun if no match

# Example usage:
text = "The dogs are barking loudly in the garden."
lemmatized_text = perform_lemmatization(text)
print("Lemmatized text:", lemmatized_text)


Lemmatized text: The dog be barking loudly in the garden .


In [40]:
data.text = data.text.apply(perform_lemmatization)

<ipython-input-40-cd2d1a7cc018>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text = data.text.apply(perform_lemmatization)


In [41]:
data.sample(100)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
180,253,115768,True,Sun Oct 29 21:02:31 +0000 2017,go pp total stuck hour prior date froze work s...,251,NaN
452,756,115868,True,Tue Oct 31 20:49:07 +0000 2017,button appl support,755,NaN
63,70,ChipotleTweets,False,Tue Oct 31 22:13:29 +0000 2017,hope get point beck,NaN,71.0
324,619,115820,True,Tue Oct 31 23:32:26 +0000 2017,amazon help frankli patienc anoth chat custom ...,NaN,618.0
40,46,115722,True,Tue Oct 31 22:04:29 +0000 2017,hello duke copi bill state servic locat nap,42,47.0
...,...,...,...,...,...,...,...
54,60,VerizonSupport,False,Tue Oct 31 22:04:29 +0000 2017,area locat servic time,NaN,61.0
314,608,115816,True,Fri Oct 27 15:48:13 +0000 2017,worth squeez get u help along way own smallbiz...,607,NaN
225,316,115785,True,Wed Nov 01 01:13:10 +0000 2017,box support kind need make sure bank would nic...,NaN,315.0
356,650,AmazonHelp,False,Tue Oct 31 22:24:47 +0000 2017,oh want hear experi plea provid feedback,651,652.0


# Tokenization

In [42]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
from nltk.tokenize import word_tokenize

def tokenize_text(text):
    # Tokenize the text using NLTK's word_tokenize function
    tokens = word_tokenize(text)
    return tokens

In [44]:
data["text"] = data["text"].apply(tokenize_text)

<ipython-input-44-402a9abe038b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(tokenize_text)


In [46]:
sample = data["text"].head(10)

In [47]:
sample

0    [understand, would, like, assist, would, need,...
1                                  [sprintcar, propos]
2    [sprintcar, sent, sever, privat, messag, one, ...
3    [plea, send, u, privat, messag, assist, click,...
4                                          [sprintcar]
5    [plea, send, u, privat, messag, gain, detail, ...
6                     [sprintcar, bad, custom, servic]
7              [madden, hear, plea, shoot, u, look, c]
8    [sprintcar, donna, magic, chang, conduct, whol...
9    [understand, concern, like, plea, send, u, dir...
Name: text, dtype: object

In [53]:
def preprocess_texts(texts):
    # Create an empty set to store unique words
    vocabulary = set()

    # Iterate through each text in the input list of texts
    for text in texts:
        # Convert the text to lowercase and split into words
        words = text.lower().split()
        # Add the words to the vocabulary set
        vocabulary.update(words)

    # Convert the set of unique words to a list and sort it
    vocabulary = sorted(list(vocabulary))

    # Create a dictionary to store the index of each word in the vocabulary
    word_to_index = {word: index for index, word in enumerate(vocabulary)}

    # Return the vocabulary and word_to_index dictionary
    return vocabulary, word_to_index


In [54]:
data["bag_words"] = data["text"].apply(preprocess_texts)

<ipython-input-54-a4e159539b7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["bag_words"] = data["text"].apply(preprocess_texts)


In [79]:
for i in range(20):
  print(data.bag_words.iloc[i])

(['assist', 'get', 'like', 'link', 'need', 'privat', 'secur', 'understand', 'would'], {'assist': 0, 'get': 1, 'like': 2, 'link': 3, 'need': 4, 'privat': 5, 'secur': 6, 'understand': 7, 'would': 8})
(['propos', 'sprintcar'], {'propos': 0, 'sprintcar': 1})
(['messag', 'one', 'privat', 'respond', 'sent', 'sever', 'sprintcar', 'usual'], {'messag': 0, 'one': 1, 'privat': 2, 'respond': 3, 'sent': 4, 'sever': 5, 'sprintcar': 6, 'usual': 7})
(['assist', 'click', 'messag', 'plea', 'privat', 'profil', 'send', 'top', 'u'], {'assist': 0, 'click': 1, 'messag': 2, 'plea': 3, 'privat': 4, 'profil': 5, 'send': 6, 'top': 7, 'u': 8})
(['sprintcar'], {'sprintcar': 0})
(['account', 'detail', 'gain', 'messag', 'plea', 'privat', 'send', 'u'], {'account': 0, 'detail': 1, 'gain': 2, 'messag': 3, 'plea': 4, 'privat': 5, 'send': 6, 'u': 7})
(['bad', 'custom', 'servic', 'sprintcar'], {'bad': 0, 'custom': 1, 'servic': 2, 'sprintcar': 3})
(['c', 'hear', 'look', 'madden', 'plea', 'shoot', 'u'], {'c': 0, 'hear': 1, 

# Advantages and disadvantages of One-hot encoding

## Advantages:
i)  One-hot encoding is straightforward to implement and understand. It does not require any complex computations or algorithms.

ii) One-hot encoding preserves the categorical information of the original data, making it suitable for algorithms that cannot directly process categorical variables.

iii) Each category is represented by its own binary feature, ensuring that the features are orthogonal (independent) from each other. This can be advantageous for some machine learning algorithms that assume feature independence.

iv)  The resulting encoded features are easily interpretable. Each binary feature indicates the presence or absence of a particular category.

## Disadvantages:

i) It can lead to a high-dimensional feature space, especially when we dealing with categorical variables with many unique categories. It can increase the computational complexity and memory requirements of models.

ii) It does not capture any ordinal relationship between categories. All categories are treated as equally dissimilar, which may not reflect the actual relationship between categories in the data.

iii) In datasets with a large number of unique categories, most of the encoded vectors will consist of zeros, resulting in a sparse representation. Sparse matrices can be inefficient in terms of storage and computation.

iv) It is designed for categorical variables and is not suitable for continuous variables. Trying to apply one-hot encoding to continuous variables may result in an explosion of dimensions and loss of information.



# Advantages and Disadvantages of bag of words (BOW)


## Advantages
i) The BOW model is simple to understand and implement. It treats each document as a collection of words, disregarding grammar and word order, making it straightforward to generate feature vectors.

ii)BOW can be applied to various natural language processing (NLP) tasks, including text classification, sentiment analysis, and information retrieval.

iii)It is computationally efficient, especially for large datasets, as it only requires counting the occurrences of each word in the document corpus.

iv)It does not rely on specific language rules or structures, making it applicable to any language.  

v)It can easily handle new words or terms that were not present in the training data by adding new dimensions to the feature vector.

## Disadvantages

i) BOW disregards the sequence and context of words within the document, leading to the loss of important semantic information. This can result in a loss of meaning, especially in tasks where word order matters, such as language generation or translation.

ii) It representation often leads to high-dimensional sparse vectors, especially when dealing with a large vocabulary or corpus. This can increase computational complexity and memory requirements, and it may require dimensionality reduction techniques.

iii) It treats each word as an independent entity and does not capture the semantic relationships between words. Consequently, it may struggle with polysemy (multiple meanings of a word) and homonymy (same spelling, different meanings) since it represents each word by a unique feature.

iv)It cannot handle out-of-vocabulary words effectively, as it only considers words seen during training. Out-of-vocabulary words are either ignored or represented as zero counts, which may lead to information loss.

v) BoW may not perform well with short texts, such as tweets or product reviews, where the context and semantics play a crucial role, but the word frequency is low.

# Advantages and disadvantages of N-grams

## Advantages of N-gram

i) N-grams preserve local word context, enabling better understanding of meaning within short segments of text.

ii) It offer flexibility in capturing different levels of linguistic information, from individual words to phrases or chunks.

iii) N-grams are widely used in language modeling tasks, facilitating accurate prediction of next words or phrases based on context.

iv) In certain applications like machine translation or speech recognition, N-grams can improve performance by incorporating contextual information.

## Disadvantages of N-gram

i) With larger values of N, the occurrence of specific N-grams decreases, leading to data sparsity issues and less reliable statistics.

ii) As N increases, the dimensionality of the feature space grows exponentially, making computation and storage more challenging.

iii) Storing and processing N-grams requires significant memory resources, particularly for large corpora or high-order N-grams.

iv) N-grams may struggle to capture long-range dependencies or syntactic structures, limiting their ability to generalize effectively across different types of text or languages.